# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [34]:
import pandas as pd
import numpy as np

from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import statsmodels.api as sm
import statsmodels.formula.api as smf

import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

Pretendo usar a biblioteca scikitlearn, portanto vou tratar os dados para remover valores nulos

In [35]:
df = pd.read_csv('previsao_de_renda.csv').iloc[:, 3:].dropna()
X = df.drop(columns='renda')
y = df['renda']
print(df.shape)
df.head()

(12427, 12)


,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,F,False,True,0,Empresário,Secundário,Solteiro,Casa,26,6.602740,1.0,8060.34
1,M,True,True,0,Assalariado,Superior completo,Casado,Casa,28,7.183562,2.0,1852.15
2,F,True,True,0,Empresário,Superior completo,Casado,Casa,35,0.838356,2.0,2253.89
3,F,False,True,1,Servidor público,Superior completo,Casado,Casa,30,4.846575,3.0,6600.77
4,M,True,False,0,Assalariado,Secundário,Solteiro,Governamental,33,4.293151,1.0,6475.97


In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12427 entries, 0 to 14999
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   sexo                   12427 non-null  object 
 1   posse_de_veiculo       12427 non-null  bool   
 2   posse_de_imovel        12427 non-null  bool   
 3   qtd_filhos             12427 non-null  int64  
 4   tipo_renda             12427 non-null  object 
 5   educacao               12427 non-null  object 
 6   estado_civil           12427 non-null  object 
 7   tipo_residencia        12427 non-null  object 
 8   idade                  12427 non-null  int64  
 9   tempo_emprego          12427 non-null  float64
 10  qt_pessoas_residencia  12427 non-null  float64
 11  renda                  12427 non-null  float64
dtypes: bool(2), float64(3), int64(2), object(5)
memory usage: 1.1+ MB


1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).

In [37]:
print(str(list(df.columns)).replace('[', '').replace(',', '').replace("'", "").replace(']', '').replace(' ', ' + \n'))

sexo + 
posse_de_veiculo + 
posse_de_imovel + 
qtd_filhos + 
tipo_renda + 
educacao + 
estado_civil + 
tipo_residencia + 
idade + 
tempo_emprego + 
qt_pessoas_residencia + 
renda


In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=.25, random_state=42)

In [43]:
df_train = pd.concat([X_train, y_train], axis=1)

2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?

In [38]:
modelo = '''np.log(renda) ~ sexo + 
            posse_de_veiculo + 
            posse_de_imovel + 
            qtd_filhos + 
            tipo_renda + 
            educacao + 
            estado_civil + 
            tipo_residencia + 
            idade + 
            tempo_emprego + 
            qt_pessoas_residencia'''

In [44]:
md = smf.ols(modelo, data=df_train)
reg = md.fit_regularized(method = 'elastic_net' 
                         , refit = True
                         , L1_wt=0
                         , alpha = 0.01)

try:
    reg.summary()
except NotImplementedError as e:
    print(f"Método não implementado")

Método não implementado


O método *.summary()* ainda não está implementado para regressão *Rigde* no *statsmodel*. Vou tentar outro recurso

In [45]:
from sklearn.metrics import r2_score

y_pred = np.exp(reg.predict(X_test))

r2 = r2_score(y_test, y_pred)

r2

0.3076345894666336

In [46]:
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]

for alpha in alphas:
    md = smf.ols(modelo, data = df_train)
    reg = md.fit_regularized(method = 'elastic_net' 
                            , refit = True
                            , L1_wt=0
                            , alpha = alpha)
    y_pred = np.exp(reg.predict(X_test))

    r2 = r2_score(y_test, y_pred)

    print(f"R² da regressão Ridge na base de testes com alpha={alpha}: {r2}\n")

R² da regressão Ridge na base de testes com alpha=0: 0.29455735164490837

R² da regressão Ridge na base de testes com alpha=0.001: 0.2996142420872713

R² da regressão Ridge na base de testes com alpha=0.005: 0.30718429599878816

R² da regressão Ridge na base de testes com alpha=0.01: 0.3076345894666336

R² da regressão Ridge na base de testes com alpha=0.05: -0.7989796297816854

R² da regressão Ridge na base de testes com alpha=0.1: -26.27898505162479



O melhor modelo até aqui parece ser o que usa alpha=0.01

3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?

In [47]:
for alpha in alphas:
    md = smf.ols(modelo, data = df_train)
    reg = md.fit_regularized(method = 'elastic_net' 
                            , refit = True
                            , L1_wt=1
                            , alpha = alpha)
    y_pred = np.exp(reg.predict(X_test))

    r2 = r2_score(y_test, y_pred)

    print(f"R² da regressão Lasso na base de testes com alpha={alpha}: {r2}\n")

R² da regressão Lasso na base de testes com alpha=0: 0.2945573516449098

R² da regressão Lasso na base de testes com alpha=0.001: 0.2947786000590946

R² da regressão Lasso na base de testes com alpha=0.005: 0.2870024250435904

R² da regressão Lasso na base de testes com alpha=0.01: 0.2866866846260483

R² da regressão Lasso na base de testes com alpha=0.05: 0.2787248413179789

R² da regressão Lasso na base de testes com alpha=0.1: 0.2787248413179789



Avaliando apenas R², o modelo Rigde com alpha=0.01 chega a um melhor resultado

4. Rode um modelo *stepwise*. Avalie o $R^2$ na base de testes. Qual o melhor resultado?

In [48]:
def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.01, 
                       threshold_out=0.05, 
                       verbose=True):
    """ Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (default is empty)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    """
    included = list(initial_list)
    while True:
        changed = False
        # forward step
        excluded = list(set(X.columns) - set(included))
        new_pval = pd.Series(index=excluded, dtype=float)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included + [new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed = True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max()
        if worst_pval > threshold_out:
            changed = True
            worst_feature = pvalues.idxmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

In [49]:
try:
    result = stepwise_selection(X, np.log(y))

    print('resulting features:')
    print(result)
except ValueError as ve:
    print(ve)

Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).


In [50]:
X.dtypes

sexo                      object
posse_de_veiculo            bool
posse_de_imovel             bool
qtd_filhos                 int64
tipo_renda                object
educacao                  object
estado_civil              object
tipo_residencia           object
idade                      int64
tempo_emprego            float64
qt_pessoas_residencia    float64
dtype: object

A função stepwise não consegue lidar diretamente com valores object ou bool. Vou fazer as conversões necessárias

In [51]:
X_step = pd.get_dummies(X, drop_first=True, dtype='int')
for coluna in X_step.columns:
    if X_step[coluna].dtype == 'bool':
        X_step[coluna] = X_step[coluna].astype('int')
X_step.head()

,posse_de_veiculo,posse_de_imovel,qtd_filhos,idade,tempo_emprego,qt_pessoas_residencia,sexo_M,tipo_renda_Bolsista,tipo_renda_Empresário,tipo_renda_Pensionista,...,educacao_Superior incompleto,estado_civil_Separado,estado_civil_Solteiro,estado_civil_União,estado_civil_Viúvo,tipo_residencia_Casa,tipo_residencia_Com os pais,tipo_residencia_Comunitário,tipo_residencia_Estúdio,tipo_residencia_Governamental
0,0,1,0,26,6.602740,1.0,0,0,1,0,...,0,0,1,0,0,1,0,0,0,0
1,1,1,0,28,7.183562,2.0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,1,1,0,35,0.838356,2.0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
3,0,1,1,30,4.846575,3.0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,1,0,0,33,4.293151,1.0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,1


In [52]:
try:
    result = stepwise_selection(X_step, np.log(y))

    print(f'\nVariáveis selecionadas:\n')
    print(result)
except ValueError as ve:
    print(ve)

Add  tempo_emprego                  with p-value 0.0
Add  sexo_M                         with p-value 0.0
Add  tipo_renda_Empresário          with p-value 1.62786e-24
Add  educacao_Superior completo     with p-value 3.98351e-16
Add  idade                          with p-value 2.09057e-13
Add  posse_de_imovel                with p-value 4.58727e-10
Add  qtd_filhos                     with p-value 0.000598764
Add  posse_de_veiculo               with p-value 0.00151995
Add  tipo_renda_Servidor público    with p-value 0.00780024

Variáveis selecionadas:

['tempo_emprego', 'sexo_M', 'tipo_renda_Empresário', 'educacao_Superior completo', 'idade', 'posse_de_imovel', 'qtd_filhos', 'posse_de_veiculo', 'tipo_renda_Servidor público']


In [53]:
X_train_step, X_test_step, y_train_step, y_test_step = train_test_split(X_step, y, test_size=.25, random_state=42)

reg_stepwise = sm.OLS(y_train_step, sm.add_constant(pd.DataFrame(X_train_step[result]))).fit()

X_test_sm = sm.add_constant(X_test_step[result])
y_pred = reg_stepwise.predict(X_test_sm)

r2_step = r2_score(y_test_step, y_pred)

print(f'R-quadrado do modelo Stepwise nos dados de teste: {r2}')

R-quadrado do modelo Stepwise nos dados de teste: 0.2787248413179789


Rodei o código para gerar o stepwise algumas vezes e o resultado é sempre o mesmo, portanto o melhor que consegui com esse método

5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?

In [54]:
md = smf.ols(modelo, data = df_train)
reg_lasso = md.fit_regularized(method = 'elastic_net' 
                        , refit = True
                        , L1_wt=1
                        , alpha = 0.001)

r2_lasso = r2_score(y_test, np.exp(reg_lasso.predict(X_test)))

print(f'R² do melhor modelo Lasso na base de testes: {r2_lasso}\n')

reg_ridge = md.fit_regularized(method = 'elastic_net' 
                        , refit = True
                        , L1_wt=0
                        , alpha = 0.01)

r2_ridge = r2_score(y_test, np.exp(reg_ridge.predict(X_test)))

print(f'R² do melhor modelo Ridge na base de testes: {r2_ridge}\n')

print(f'R² do melhor modelo Stepwise na base de testes: {r2_step}')

R² do melhor modelo Lasso na base de testes: 0.2947786000590946

R² do melhor modelo Ridge na base de testes: 0.3076345894666336

R² do melhor modelo Stepwise na base de testes: 0.2977549927961066


Avaliando apenas o R², o melhor modelo parece ser o Ridge

6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.

In [18]:
print(modelo)

np.log(renda) ~ sexo + 
            posse_de_veiculo + 
            posse_de_imovel + 
            qtd_filhos + 
            tipo_renda + 
            educacao + 
            estado_civil + 
            tipo_residencia + 
            idade + 
            tempo_emprego + 
            qt_pessoas_residencia


In [123]:
modelo_ajustado = '''
np.log(renda) ~ sexo + 
            posse_de_veiculo + 
            posse_de_imovel + 
            tipo_renda + 
            educacao + 
            estado_civil + 
            tipo_residencia +
            standardize(qtd_filhos) +
            idade + 
            np.sqrt(tempo_emprego) + 
            center(qt_pessoas_residencia)
'''

In [124]:
md_ajustado = smf.ols(modelo_ajustado, data = df_train)
reg_lasso = md_ajustado.fit_regularized(method = 'elastic_net' 
                        , refit = True
                        , L1_wt=1
                        , alpha = 0.001)

r2_lasso = r2_score(y_test, np.exp(reg_lasso.predict(X_test)))

print(f'R² do melhor modelo Lasso na base de testes: {r2_lasso}\n')

reg_ridge = md_ajustado.fit_regularized(method = 'elastic_net' 
                        , refit = True
                        , L1_wt=0
                        , alpha = 0.01)

r2_ridge = r2_score(y_test, np.exp(reg_ridge.predict(X_test)))

print(f'R² do melhor modelo Ridge na base de testes: {r2_ridge}\n')

R² do melhor modelo Lasso na base de testes: 0.2667375163930933

R² do melhor modelo Ridge na base de testes: 0.32075041032226914



Consegui melhorar um pouco o valor de R² para a regressão Ridge, porém piorou na Lasso. Vou tentar outra abordagem

In [152]:
modelo_ajustado_2 = '''
np.log(renda) ~ sexo + 
            posse_de_veiculo + 
            posse_de_imovel + 
            tipo_renda + 
            educacao + 
            estado_civil + 
            tipo_residencia +
            qtd_filhos +
            np.log(idade) + 
            tempo_emprego + 
            qt_pessoas_residencia
'''

In [153]:
md_ajustado_2 = smf.ols(modelo_ajustado_2, data = df_train)
reg_lasso = md_ajustado_2.fit_regularized(method = 'elastic_net' 
                        , refit = True
                        , L1_wt=1
                        , alpha = 0.001)

r2_lasso = r2_score(y_test, np.exp(reg_lasso.predict(X_test)))

print(f'R² do melhor modelo Lasso na base de testes: {r2_lasso}\n')

reg_ridge = md_ajustado_2.fit_regularized(method = 'elastic_net' 
                        , refit = True
                        , L1_wt=0
                        , alpha = 0.01)

r2_ridge = r2_score(y_test, np.exp(reg_ridge.predict(X_test)))

print(f'R² do melhor modelo Ridge na base de testes: {r2_ridge}\n')

R² do melhor modelo Lasso na base de testes: 0.29511564060734274

R² do melhor modelo Ridge na base de testes: 0.30184048826654486



Consegui uma melhora bem pequena no Lasso, aplicando o log na idade no modelo

7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [156]:
reg_tree = DecisionTreeRegressor()
reg_tree.fit(X_train_step, y_train_step)

print(f"R² da árvore de regressão: {reg_tree.score(X_test_step, y_test_step)}")

R² da árvore de regressão: 0.23210037063126965


Resultado foi inferior